Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 файлам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import datetime
import calendar

Мой импорт файлов

In [3]:
folder_path = ''

sales = pd.read_csv(folder_path+'sales_train.csv')
sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')
items = pd.read_csv(folder_path+'items.csv')
categ = pd.read_csv(folder_path+'item_categories.csv')
shops = pd.read_csv(folder_path+'shops.csv')

Исходный импорт файлов

In [3]:
from google.colab import drive
drive.mount('/content/drive')

categ = pd.read_csv("/content/drive/MyDrive/ITMO.Work/21 22/ВИМ/data/item_categories.csv")
items = pd.read_csv("/content/drive/MyDrive/ITMO.Work/21 22/ВИМ/data/items.csv")
sales = pd.read_csv("/content/drive/MyDrive/ITMO.Work/21 22/ВИМ/data/sales_train.csv")
sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')
shops = pd.read_csv("/content/drive/MyDrive/ITMO.Work/21 22/ВИМ/data/shops.csv")

In [4]:
sales[sales['date_block_num'] == 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
115685,2013-01-19,0,46,32,349.00,1.0
115686,2013-01-18,0,46,32,349.00,1.0
115687,2013-01-26,0,46,35,399.00,2.0
115688,2013-01-31,0,46,621,149.00,1.0


In [5]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [6]:
items.head()

,item_name,item_id,category_id
0,!! IN THE POWER OF HAPPINESS (PLAST) D,0,40
1,! ABBYY FineReader 12 Professional Edition Ful...,1,76
2,*** IN THE GLORY OF THE GLORY (UNV) D,2,40
3,*** BLUE WAVE (Univ) D,3,40
4,*** BOX (GLASS) D,4,40


In [7]:
categ.head()

,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4


In [8]:
shops.head()

,shop_name,shop_id
0,"! Yakutsk Ordzhonikidze, 56 francs",0
1,"! Yakutsk TC ""Central"" fran",1
2,"Adygea TC ""Mega""",2
3,"Balashikha TC ""Oktyabr-Kinomir""",3
4,"Volga TC ""Volga Mall""",4


# Задание 1 (3 балла)

1. Возьмите 10 000 случайных строк из таблицы sales_train и найдите в них долю товаров из категории 37. 
2. Найдите самый удачный по общему количеству продаж день для магазина St. Petersburg Nevsky Center shopping center за 2014 год.
3. Есть ли товар, который продается успешнее всех остальных (по ежемесячному количеству) во всех магазинах?

1

In [9]:
rows = sales.sample(10000, random_state=42)
item_id = rows.merge(items, on='item_id')['category_id']
(item_id == 37).mean()

0.065

2

In [6]:
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


In [7]:
shops

,shop_name,shop_id
0,"! Yakutsk Ordzhonikidze, 56 francs",0
1,"! Yakutsk TC ""Central"" fran",1
2,"Adygea TC ""Mega""",2
3,"Balashikha TC ""Oktyabr-Kinomir""",3
4,"Volga TC ""Volga Mall""",4
5,"Vologda SEC ""Marmelad""",5
6,"Voronezh (Plekhanovskaya, 13)",6
7,"Voronezh SEC ""Maksimir""",7
8,"Voronezh SEC City-Park ""Grad""",8
9,Outbound Trade,9


In [5]:
sales.merge(shops, on='shop_id')

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name
0,2013-01-02,0,59,22154,999.0,1.0,"Yaroslavl TC"" Altair """
1,2013-01-10,0,59,22151,399.0,1.0,"Yaroslavl TC"" Altair """
2,2013-01-04,0,59,5603,699.0,1.0,"Yaroslavl TC"" Altair """
3,2013-01-19,0,59,5587,199.0,2.0,"Yaroslavl TC"" Altair """
4,2013-01-31,0,59,5613,5571.0,1.0,"Yaroslavl TC"" Altair """
...,...,...,...,...,...,...,...
2935844,2015-10-30,33,36,20231,169.0,1.0,"Novosibirsk SEC"" Gallery Novosibirsk """
2935845,2015-10-18,33,36,20334,1949.0,1.0,"Novosibirsk SEC"" Gallery Novosibirsk """
2935846,2015-10-31,33,36,20339,1549.0,1.0,"Novosibirsk SEC"" Gallery Novosibirsk """
2935847,2015-10-31,33,36,20352,1549.0,1.0,"Novosibirsk SEC"" Gallery Novosibirsk """


In [10]:
df = sales.merge(shops, on='shop_id')
df['year'] = pd.to_datetime(df['date']).dt.year
mask_1 = df['shop_name'].str.contains('St. Petersburg Nevsky Center shopping center')
mask_2 =  df['year'] == 2014
df = df[mask_1 & mask_2]
(df.groupby(by='date')['item_cnt_day'].sum()).idxmax()

Timestamp('2014-12-27 00:00:00')

3

**Как я понял это задание**

В каждом месяце у магазина есть лучший товар по продажам. 

Если для каждого магазина каждый месяц лучший товар один и тот же, то ответ да. 

Если хотя бы в одном магазине лучшие товары отличаютс, то ответ нет

In [11]:
def get_best_item(x):
  df = x.groupby('item_id')['item_cnt_day'].sum().idxmax()
  return df


df = sales
res = df.groupby(['shop_id', 'date_block_num']).apply(get_best_item).reset_index(name='best_item')
nunique = res.groupby('shop_id').apply(lambda x: x['best_item'].nunique())
(nunique == 1).all()

False

In [12]:
res

,shop_id,date_block_num,best_item
0,0,0,14433
1,0,1,14752
2,1,0,13351
3,1,1,13354
4,2,0,2808
...,...,...,...
1581,59,29,20949
1582,59,30,17717
1583,59,31,6507
1584,59,32,4719


# Задание 2 (2 балла)

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

1

In [13]:
id_bought = sales.groupby('item_id')['item_cnt_day'].sum().reset_index(name='sm_bought')
top10 = id_bought.sort_values('sm_bought', ascending=False).head(10).reset_index(drop=True)
top10

,item_id,sm_bought
0,20949,187642.0
1,2808,17245.0
2,3732,16642.0
3,17717,15830.0
4,5822,14515.0
5,3734,11688.0
6,6675,10289.0
7,3731,10099.0
8,1855,10032.0
9,16787,9227.0


In [14]:
id_bought = sales.groupby('item_id')['item_cnt_day'].sum().reset_index(name='sm_bought')
top10 = id_bought.sort_values('sm_bought', ascending=False).head(10).reset_index(drop=True)
top10 = top10['item_id']
top10

0    20949
1     2808
2     3732
3    17717
4     5822
5     3734
6     6675
7     3731
8     1855
9    16787
Name: item_id, dtype: int64

2

In [15]:
def is_saled(item_id):
  expected_years = np.array([2013, 2014, 2015])
  mask = sales['item_id'] == item_id
  sale_years = sales.loc[mask, 'date'].dt.year.unique()
  intersect = np.intersect1d(expected_years, sale_years)
  return intersect.size == 3

mask = top10.transform(is_saled)
top10[mask]

0    20949
1     2808
2     3732
3    17717
4     5822
5     3734
6     6675
8     1855
9    16787
Name: item_id, dtype: int64

# Задание 3 (3 балла)

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

*Если таких категорий несколько, выведите все.*

1

In [16]:
category_sum = defaultdict(lambda: 0)

bought_sum = sales.groupby('item_id')['item_cnt_day'].sum()

for row in items[['item_id', 'category_id']].itertuples():
  item_id = row[1]
  category = row[2]
  category_sum[category] += bought_sum.get(item_id, 0)

category_sum = pd.Series(category_sum)

In [17]:
categ

,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4
...,...,...
79,Service,79
80,Service - Tickets,80
81,Blank media (spire),81
82,Blank media (piece),82


In [18]:
def output_category(_id):
  mask = categ['item_category_id'] == _id
  return categ.loc[mask]

In [19]:
category_sum.sort_values(ascending=True)

51         1.0
10         1.0
1          2.0
0          3.0
50         3.0
        ...   
37    203284.0
19    254887.0
55    348591.0
30    456540.0
40    634171.0
Length: 84, dtype: float64

In [20]:
min_indexes = category_sum[category_sum == category_sum.min()].index
min_category_id1 = min_indexes[0]
min_category_id2 = min_indexes[1]

In [21]:
output_category(min_category_id1)

,item_category_name,item_category_id
10,Game consoles - PS2,10


In [22]:
output_category(min_category_id2)

,item_category_name,item_category_id
51,Books - Cognitive literature,51


In [23]:
max_category_id = category_sum.idxmax()
output_category(max_category_id)

,item_category_name,item_category_id
40,Cinema - DVD,40


# Задание 4 (3 балла)

1. Для категорий из задания 3 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами. Оставьте в таблице только те товары, среднее количество продаж которых строго больше 100.
2. Найдите месяц с максимальными продажами для каждого товара из пункта 1.


In [24]:
def get_only_category(category_id):
  category_mask = items['category_id'] == category_id
  only_category_items = items.loc[category_mask, ['item_id']]
  only_category = pd.merge(sales, only_category_items, on='item_id')
  return only_category

In [25]:
def get_items_mean_lg_100( category_id):
  # получаем топ 5 магазинов по продажам для категории
  only_category = get_only_category(category_id)
  shop_bought = only_category.groupby('shop_id')['item_cnt_day'].sum()
  top5_shops = shop_bought.nlargest(5).index
  # получаем продажи товаров категории только в топ 5
  mask = only_category['shop_id'].isin(top5_shops)
  df_only_top5 = only_category[mask]
  # получаем предметы, у которых больше 100 в среднем по топ 5 магазинов
  item_shop = df_only_top5.groupby(['item_id', 'shop_id'])['item_cnt_day'].sum()
  item_mean = item_shop.groupby(level=0).mean()
  res = list(item_mean[item_mean > 100].index)
  return res

In [26]:
gt_100_item_id = []
gt_100_min1 = get_items_mean_lg_100(min_category_id1)
gt_100_min2 = get_items_mean_lg_100(min_category_id2)
gt_100_max = get_items_mean_lg_100(max_category_id)
gt_100_item_id.extend(gt_100_min1 + gt_100_min2 + gt_100_max)
gt_100_item_id = pd.Series(gt_100_item_id)
gt_100_item_id

0        30
1        32
2      8415
3      9781
4      9862
5      9983
6     10292
7     10298
8     10376
9     10487
10    10497
11    10956
12    10991
13    11457
14    11496
15    11921
16    11922
17    11966
18    12020
19    12361
20    12362
21    12734
22    13811
23    14019
24    14305
25    14447
26    14574
27    14730
28    16450
29    16454
30    16501
31    16581
32    17782
33    18622
34    19065
35    19116
36    19196
37    19771
38    19811
39    20088
40    20231
41    21376
42    21377
43    21384
44    21404
45    21427
46    21440
47    22020
48    22058
dtype: int64

2

In [40]:
def get_best_month(item_id):
  df_item = sales[sales['item_id'] == item_id]
  return df_item.groupby('date_block_num')['item_cnt_day'].sum().idxmax()

best_month = gt_100_item_id.transform(get_best_month)
pd.DataFrame({'item_id': gt_100_item_id,
              'best_month': best_month})

,item_id,best_month
0,30,1
1,32,0
2,8415,14
3,9781,8
4,9862,14
5,9983,9
6,10292,8
7,10298,10
8,10376,12
9,10487,11


Проверю, что результата логичен

In [41]:
item_id = 9862
df_item = sales[sales['item_id'] == item_id]
df_item.groupby('date_block_num')['item_cnt_day'].sum()

date_block_num
13     90.0
14    920.0
15    183.0
16     64.0
17     32.0
18     65.0
19     83.0
20     61.0
21     36.0
22     29.0
23     14.0
24     11.0
25      4.0
26     13.0
27     18.0
28     15.0
29      7.0
30      7.0
31      7.0
32     12.0
33     13.0
Name: item_cnt_day, dtype: float64

# Задание 5 (3 балла)

Пункт 1: «Для категорий из задания 3 найдите среднюю и медианную цену по (т.е. для каждого товара категории отдельно) категории в пределах каждого месяца (в котором товар продавался). Выведите топ-10 товаров по медиане из каждой категории (для каждого месяца).

1. Для категорий из задания 3 найдите среднюю и медианную цену по категории в пределах каждого месяца. Выведите топ-10 товаров по медиане из каждой категории.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

## 1

In [31]:
a = None


def get_top10_by_median(category_id):
  def top10_from_grouped(x):
    x = x.groupby('item_id')['item_price'].agg(['mean', 'median'])
    return x.nlargest(10, 'median')

  only_category = get_only_category(category_id)
  return only_category.groupby(['date_block_num']).apply(top10_from_grouped)

In [32]:
a

In [33]:
get_top10_by_median(max_category_id)

mean  median
date_block_num item_id                    
0              12686    958.285714   999.0
               13993    999.000000   999.0
               9601     849.000000   849.0
               9618     802.000000   849.0
               9640     849.000000   849.0
...                            ...     ...
33             14711    699.000000   699.0
               20214    649.000000   649.0
               12686    574.000000   574.0
               8206     549.000000   549.0
               11921    546.317073   549.0

[340 rows x 2 columns]

In [34]:
get_top10_by_median(min_category_id1)

,,mean,median
date_block_num,item_id,,
3,6669,1998.0,1998.0


In [35]:
get_top10_by_median(min_category_id2)

,,mean,median
date_block_num,item_id,,
18,17703,129.0,129.0


## 2

В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [42]:
# means = get_top10_by_median(max_category_id)['mean']

means = sales.groupby('date_block_num')['item_price'].mean()

mean_center = means.iloc[1:-1].reset_index(drop=True)
mean_left = means.iloc[:-2].reset_index(drop=True)
mean_right = means.iloc[2:].reset_index(drop=True)
is_local_min = (mean_center < mean_left) & (mean_center < mean_right)
is_local_min

0     False
1     False
2     False
3      True
4     False
5      True
6     False
7     False
8     False
9     False
10    False
11     True
12    False
13    False
14     True
15    False
16    False
17     True
18    False
19    False
20    False
21    False
22    False
23    False
24     True
25    False
26    False
27    False
28    False
29    False
30     True
31    False
Name: item_price, dtype: bool

In [43]:
def add_months(sourcedate, months):
  month = sourcedate.month - 1 + months
  year = sourcedate.year + month // 12
  month = month % 12 + 1
  day = min(sourcedate.day, calendar.monthrange(year,month)[1])
  return datetime.date(year, month, day)

def get_year_by_date_block(date_block):
  start_date = datetime.date(year=2013, day=1, month=1)
  actual_date = add_months(start_date, date_block)
  return actual_date.year

date_blocks = pd.Series(range(1, is_local_min.shape[0]+1))
date_blocks = date_blocks[is_local_min]
years = date_blocks.transform(get_year_by_date_block)
pd.DataFrame({'date_block': date_blocks, 'year': years})

,date_block,year
3,4,2013
5,6,2013
11,12,2014
14,15,2014
17,18,2014
24,25,2015
30,31,2015
